# 0) Looking thought two tables

In [ ]:
# Project: Analyzing Customer Churn

# In this final project you will analyze the customer churn of a telecommunications company.
# In this project you will do the following by yourself:


# `churn_data` table:

# Column number | Column name             | Type                  | Description
#  ------------ | :---------:             | :---------:           | ------------:
# 0             | `account_length`        | numerical (`int`)     | Unknown units of time, how long the customer has been a customer.
# 1             | `international_plan`    | categorical (nominal) | Contract with special conditions for cheaper calls to other countries.
# 2             | `voice_mail_plan`       | categorical (nominal) | Contract with special conditions for more voicemail storage.
# 3             | `number_vmail_messages` | numerical (`int`)     | Number of voicemail messages.
# 4             | `total_day_minutes`     | numerical (`float`)   | Duration in minutes of all calls from 8am to 4pm.
# 5             | `total_day_calls`       | numerical (`int`)     | Number of all calls from 8am to 4pm.
# 6             | `total_day_charge`      | numerical (`float`)   | Calculated costs for all calls from 8am to 4pm.
# 7             | `total_eve_minutes`     | numerical (`float`)   | Duration in minutes of all calls from 4pm to 10pm.
# 8             | `total_eve_calls`       | numerical (`int`)     | Number of all calls from 4pm to 10pm.
# 9             | `total_eve_charge`      | numerical (`float`)   | Calculated costs for all calls from 4pm to 10pm.
# 10            | `total_night_minutes`   | numerical (`float`)   | Duration in minutes of all calls from 10pm to 8am.
# 11            | `total_night_calls`     | numerical (`int`)     | Number of all calls from 10pm to 8am.
# 12            | `total_night_charge`    | numerical (`float`)   | Calculated costs for all calls from 10pm to 8am.
# 13            | `customer_service_calls`| numerical (`int`)     | Number of calls to customer service, e.g. due to technical problems.
# 14            | `churn`                 | categorical (nominal) | Did the customer leave? (1=yes 0=no)
# 15            | `local_area_code`       | categorical (nominal) | local area code for telephone.
# 16            | `phone_num`             | Categorical (nominal) | Customers telephone number not including the local area code.

# `cities` table:

# Column number | Column name       | Type                  | Description
#  ------------ | :---------:       |:---------:            | ------------:
# 0             | `city`            | categorical (nominal) | Cities.
# 1             | `area_code`       | categorical (nominal) | local area code for telephone.



# 1 ) Importing data

In [ ]:
#import the required libraries
import sqlalchemy as sa
import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
%matplotlib inline

#SQL query to join the tables
engine = sa.create_engine('sqlite:///telco_churn.db')
connection = engine.connect()

#check which tables the document has 
inspector = sa.inspect(engine)
table_names = inspector.get_table_names()

table_names

In [ ]:
#create new dataFrame to put all values from first table
df_1 = pd.read_sql('SELECT * FROM churn_data', connection)
df_1.head()

#create new dataFrame to put all values from second table
df_2 = pd.read_sql('SELECT * FROM cities', connection)
df_2.head()

#union two tables together
query_string = '''SELECT * 
FROM churn_data
JOIN cities
ON churn_data.local_area_code = cities.area_code'''
df = pd.read_sql(query_string, connection)
df

# 2) Check and clean the data

In [ ]:
#drop the unnecessary column "local_area_code", as column "area_code" has the same values
df = df.drop('local_area_code', axis =1)
df

In [ ]:
#use command "describe" to inspect the numerical data
df.describe()

In [ ]:
#create the mask to drop the wrong values in the column "customer_service_calls"
mask_customer = df.loc[:, 'customer_service_calls'] <0
df.loc[mask_customer,'customer_service_calls'] = np.nan
df= df.dropna(axis = 0, thresh=18)

#check the shape 
df.shape

In [ ]:
#check the types of data 
df.dtypes

In [ ]:
#change the types of data

df.loc[:, 'account_length'] = df.loc[:, 'account_length'].astype('float32')
df.loc[:, 'number_vmail_messages'] = df.loc[:, 'number_vmail_messages'].astype('float32')
df.loc[:, 'total_day_minutes'] = df.loc[:, 'total_day_minutes'].astype('float32')
df.loc[:, 'total_day_calls'] = df.loc[:, 'total_day_calls'].astype('float32')
df.loc[:, 'total_day_charge'] = df.loc[:, 'total_day_charge'].astype('float32')
df.loc[:, 'total_eve_minutes'] = df.loc[:, 'total_eve_minutes'].astype('float32')
df.loc[:, 'total_eve_calls'] = df.loc[:, 'total_eve_calls'].astype('float32')
df.loc[:, 'total_night_minutes'] = df.loc[:, 'total_night_minutes'].astype('float32')
df.loc[:, 'total_night_calls'] = df.loc[:, 'total_night_calls'].astype('float32')
df.loc[:, 'total_night_charge'] = df.loc[:, 'total_night_charge'].astype('float32')

df.loc[:, 'customer_service_calls'] = df.loc[:, 'customer_service_calls'].astype('float32')
df.loc[:, 'phone_num'] = df.loc[:, 'phone_num'].astype('float32')

df.loc[:, 'churn'] = df.loc[:, 'churn'].astype('int32')
df.loc[:, 'area_code'] = df.loc[:, 'area_code'].astype('int32')

df.loc[:, 'international_plan'] = df.loc[:, 'international_plan'].astype('category')
df.loc[:, 'voice_mail_plan'] = df.loc[:, 'voice_mail_plan'].astype('category')
df.loc[:, 'city'] = df.loc[:, 'city'].astype('category')
df.dtypes

In [ ]:
#check if the values are null in the table 
df.isna().sum()

In [ ]:
#the correlation matrix

fig, ax = plt.subplots(figsize = [20,20])
sns.heatmap(df.corr(), annot=True, ax =ax)

# 3) What are the names of the four cities with the highest rates of customer churn?

In [ ]:
#identify four cities with the highest customer churn
four_cities  = pd.crosstab(index = df.loc[:, 'city'], columns = df.loc[:, 'churn']).sort_values(by = 1, ascending= False) 
our_four_cities = four_cities[:4]
our_four_cities

# 4a) Which categorical data series should be used to identify customers who will possibly leave soon? Which customers should be contacted based on this data series?

In [ ]:
#сheck the column "international plan" 


inter1 = pd.crosstab(index  = df.loc[:, 'churn'], columns = df.loc[:, 'international_plan'])
print(inter1)

inter2 = pd.crosstab(index  = df.loc[:, 'churn'], columns = df.loc[:, 'international_plan'], normalize= 'columns')
print(inter2)

In [ ]:
# check the column "voice_mail_plan"

voice = pd.crosstab(index  = df.loc[:, 'voice_mail_plan'], columns = df.loc[:, 'churn'])
voice

In [ ]:
#a lot of customers have the extra voicemail package, but these customers don't look as likely to leave.

#more than 40% of customers who had the international plan, left the company. it looks very suspicious,
# that's why we need to look at these clients

In [ ]:
mask_for_categor1 = df.loc[:,  'international_plan'] == 'yes'
mask_for_categor2 = df.loc[:, 'churn'] ==0
mask_for_all  = mask_for_categor1 & mask_for_categor2
df_cat = df.loc[mask_for_all, :]
df_cat.head()

# 4b) Which integer data series would you also use for this and how would you set the threshold? Which customers should be contacted based on this data series?

In [ ]:
#check the amount of unique data in columns with integer data 

count_of_email = len(df.loc[:, "number_vmail_messages"].unique())
print(count_of_email)

count_of_day_calls = len(df.loc[:, "total_day_calls"].unique())
print(count_of_day_calls)

count_of_eve_calls = len(df.loc[:, "total_eve_calls"].unique())
print(count_of_eve_calls)

count_of_night_calls = len(df.loc[:, "total_night_calls"].unique())
print(count_of_night_calls)

count_service = len(df.loc[:, "customer_service_calls"].unique())
print(count_service)

In [ ]:
#the column "customer_service_calls" should be treated as categorical
#checking column "customer_service_calls" with histograms
service = pd.crosstab(index  = df.loc[:, 'customer_service_calls'], columns = df.loc[:, 'churn'])
service
service.plot(kind = 'bar')

In [ ]:
#checking the columns 'total_day_calls', 'total_eve_calls' and 'total_night_calls' with histograms 

fig, ax1 = plt.subplots(nrows = 3, figsize =[10,20])
df.groupby('churn')['total_day_calls'].plot(kind='hist',legend=True, ax = ax1[0])
df.groupby('churn')['total_eve_calls'].plot(kind='hist',legend=True, ax = ax1[1])
df.groupby('churn')['total_night_calls'].plot(kind='hist',legend=True, ax = ax1[2])

In [ ]:
#checking the columns 'total_day_calls', 'total_eve_calls' and 'total_night_calls' with boxplots

fig, ax2 = plt.subplots(nrows = 1, ncols = 3)
df.boxplot(column='total_day_calls', by='churn', ax=ax2[0])
df.boxplot(column='total_eve_calls', by='churn', ax=ax2[1])
df.boxplot(column='total_night_calls', by='churn', ax=ax2[2])

In [ ]:
#checking the column 'account_length' with histogram and boxplot 

fig, ax3 = plt.subplots(nrows = 1, ncols = 2)
df.boxplot(column='account_length', by='churn', ax=ax3[0])
df.groupby('churn')['account_length'].plot(kind='hist',legend=True, ax = ax3[1])

In [ ]:
#checking the column 'number_vmail_messages' with histogram and boxplot 

fig, ax4 = plt.subplots(nrows = 1, ncols = 2)
df.boxplot(column='number_vmail_messages', by='churn', ax=ax4[0])
df.groupby('churn')['number_vmail_messages'].plot(kind='hist',legend=True, ax = ax4[1])

In [ ]:
#based on the data from the column "customer_service_calls" we can say that there is a dependence between 
#the number of customers' calls and churn statistic 

mask_for_categor2 = df.loc[:, 'churn'] == 0
mask_for_customer = df.loc[:, 'customer_service_calls'] >3
mask_for_int = mask_for_categor2 & mask_for_customer
df_int = df.loc[mask_for_int, :]
print(df_int.shape[0])

# 4c) Which floating point data series could you use to help with this selection? Determine the threshold for this by using logistic regression. Which customers should be contacted based on this data series?

In [ ]:
# make a plot with one column on the x-axis and the other on the y-axis
fig, axs = plt.subplots(nrows=1, ncols=3, figsize=[18, 6])  # define a plot with 3 Axes (day, eve, night)
df.plot(kind='scatter', 
        x='total_day_minutes', 
        y='total_day_charge', 
        ax=axs[0],
        xlim=(0,500),
        ylim=(0,60));  
df.plot(kind='scatter', 
        x='total_eve_minutes', 
        y='total_eve_charge', 
        ax=axs[1],
        xlim=(0,500),
        ylim=(0,60));  
df.plot(kind='scatter', 
        x='total_night_minutes', 
        y='total_night_charge', 
        ax=axs[2],
        xlim=(0,500),
        ylim=(0,60));  

In [ ]:
#make the mask to drop the outliers 

mask_total = df.loc[:, 'total_day_minutes'] >360
df =df.drop(df.index[mask_total], axis = 0)

In [ ]:
#checking the column 'total_day_minutes', 'total_eve_minutes' and 'total_night_minutes' with histograms and boxplots 

fig, ax5 = plt.subplots(nrows = 2,ncols=3, figsize= [12,12])
df.groupby('churn')['total_day_minutes'].plot(kind='hist',legend=True, ax =ax5[0][0], bins = 50)
df.groupby('churn')['total_eve_minutes'].plot(kind='hist',legend=True, ax = ax5[0][1],bins = 50)
df.groupby('churn')['total_night_minutes'].plot(kind='hist',legend=True, ax = ax5[0][2],bins = 50)
df.boxplot(column='total_day_minutes', by='churn', ax=ax5[1][0])
df.boxplot(column='total_eve_minutes', by='churn', ax=ax5[1][1])
df.boxplot(column='total_night_minutes', by='churn', ax=ax5[1][2])

In [ ]:
#checking the column 'total_day_charge', 'total_eve_charge' and 'total_night_charge' with histograms and boxplots

fig, ax6 = plt.subplots(nrows = 2,ncols=3, figsize= [12,12])
df.groupby('churn')['total_day_charge'].plot(kind='hist',legend=True, ax =ax6[0][0], bins = 50)
df.groupby('churn')['total_eve_charge'].plot(kind='hist',legend=True, ax = ax6[0][1],bins = 50)
df.groupby('churn')['total_night_charge'].plot(kind='hist',legend=True, ax = ax6[0][2],bins = 50)
df.boxplot(column='total_day_charge', by='churn', ax=ax6[1][0])
df.boxplot(column='total_eve_charge', by='churn', ax=ax6[1][1])
df.boxplot(column='total_night_charge', by='churn', ax=ax6[1][2])

In [ ]:
#plot pairwise relationships between variables within a dataset

df_logic_data = df.loc[:,["total_day_charge", "total_eve_charge", "total_night_charge", "total_day_minutes","total_eve_minutes", "total_night_minutes", "churn"]]
sns.pairplot(df_logic_data)

In [ ]:
#the correlation matrix which contains the floating point data
sns.heatmap(df_logic_data.corr(), annot=True)

In [ ]:
#logistic regression of churn and total_day_minutes

model = smf.logit(formula='churn ~ total_day_minutes ', data=df_logic_data)
results = model.fit()
results.summary()

In [ ]:
# find the trained results to predict a given range and plot the S-curve

X = pd.Series(range(800))  
X_df = pd.DataFrame(X)
X_df.columns = ['total_day_minutes']


p_y = results.predict(X_df)
fig, ax = plt.subplots()
p_y.plot(ax=ax)
ax.set(xlabel='total_day_minutes', ylabel='churn')

p_y[p_y>=0.5].index[0]

In [ ]:
#Calculate treshold value and plot it as a vertical line in logistic regression plot
fig, ax = plt.subplots()
ax.plot(p_y);
ax.set(xlabel='total_day_charge', ylabel='churn')
ax.axvline(p_y[p_y>=0.5].index[0], color='black')

In [ ]:
#find the customer to be contacted based on critical value from logistic regression

mask = df.loc[:,'total_day_minutes'] >= (p_y[p_y>=0.5].index[0])
df.loc[mask,:]

In [ ]:
#plot the histogram of our values and the treshold value as a vertical line
fig, ax = plt.subplots(figsize=[8, 8])
df.groupby('churn')['total_day_minutes'].plot(kind='hist', bins=50, ax=ax)
ax.set(xlim=[0, 800])
ax.vlines(x=349, ymin=0, ymax=160)

#  5) Visualizing the cities and other selected data series

In [ ]:
#we build a schedule of cities and their level of customer churn. 
#With the help of a vertical line, we separate the first four cities 
#that we should draw attention to, as they are the ones with the greatest loss of customers
from matplotlib.patches import FancyArrowPatch

fig, ax = plt.subplots(figsize = [20,10])
plt.style.use('fivethirtyeight')
my_colors = ["#FBCEB1","#A57164"]
df_city = pd.crosstab(index=df.loc[:, 'city'],columns=df.loc[:, 'churn'],normalize='index')
df_city = df_city.sort_values(by=1, ascending=False)
df_city.plot(kind='bar',ax=ax, color = my_colors)
df_city.iloc[:,1].plot(ax =ax,color='brown')

ax.set( ylim =[0,1.14], ylabel = "The percentage of churn for cities")
ax.set_title(label ="Statistics of churn for four cities",fontweight = 'bold', fontsize = 40)
ax.xaxis.set_tick_params(labelrotation=0)
ax.axvline(3.5, ymax = 0.83, color='black')

ax.legend(['Avg line',"Line of four cities", 'Clients did not leave', 'Clients left'], title= "Churn",prop = {'size':15}, bbox_to_anchor=(1.0,1.08))


first_row = ax.annotate(s = '',
                 xy=(-0.1, 0.9),
                 xytext=(-0.1, 1.05),
                arrowprops=dict(facecolor='black'))
second_row = ax.annotate(s = '',
                 xy=(0.9, 0.9),
                 xytext=(0.9, 1.05),
                arrowprops=dict(facecolor='black'))
third_row = ax.annotate(s = '',
                 xy=(1.9, 0.9),
                 xytext=(1.9, 1.05),
                arrowprops=dict(facecolor='black'))
fourth_row = ax.annotate(s = '',
                 xy=(2.9, 0.9),
                 xytext=(2.9, 1.05),
                arrowprops=dict(facecolor='black'))

In [ ]:
#these pie charts show the dependence of a categorical variable "international_plan" on customer churn.

fig, ax = plt.subplots(nrows= 1, ncols = 2, figsize =[12,10])

explode1 = (0, 0.2)
explode2 = (0, 0.07)
df_inter_var = pd.crosstab(index=df.loc[:, 'churn'],
                            columns=df.loc[:, 'international_plan'], normalize='columns')
df_inter_var.plot(kind='pie',y=0,ax=ax[0], legend = False, explode = explode1, colors = my_colors)
ax[0].set(ylabel="")
ax[0].set_title(label ='Statistics for people that \ndid not have an international plan', fontweight="bold")
df_inter_var.plot(kind='pie',y=1,ax=ax[1], legend = False, explode = explode2,colors = my_colors)
ax[1].set(ylabel= "")
ax[1].set_title(label = 'Statistics for people that \nhad an international plan', fontweight="bold")
legend = plt.legend(['Clients did not leave', 'Clients left'], 
                    prop = {'size':10}, loc='upper right', 
                    bbox_to_anchor=(0.2, 0.1), fancybox = True, 
                    shadow = True,framealpha=1, borderpad=1)

In [ ]:
#these two graphs show the dependence of number of customer's calls on churn.
#it is seen that the situation worsens when the number of calls exceeds 3

mask_for_customer = df.loc[:, 'customer_service_calls'] >3
df_int = df.loc[mask_for_int, :]

fig, ax = plt.subplots(nrows =1, ncols=2, figsize =[20,8])
df_integer = pd.crosstab(index=df.loc[:, 'customer_service_calls'],
                            columns=df.loc[:, 'churn'],
                            normalize='index'
                           )
df_integer.plot(kind='bar',ax=ax[0], legend = False, color = my_colors)
ax[0].set_ylabel(ylabel = "The percentage of churn", fontsize = 22)
ax[0].set_title(label = "General statistics for all possible \nvalues of 'customer service calls' ", loc = "center", pad = 35 ,fontweight="bold")
ax[0].xaxis.set_tick_params(labelrotation=0)
ax[0].set_xlabel(xlabel = 'customer_service_calls', fontsize = 22)

df_integer_3 = pd.crosstab(index=df.loc[mask_for_customer, 'customer_service_calls'],
                            columns=df.loc[:, 'churn'],
                           normalize='index')
df_integer_3.plot(kind='bar',ax=ax[1], legend = False, color = my_colors)
ax[1].set_yticklabels([None])
ax[1].set_ylabel(ylabel = "The percentage of churn", fontsize = 22)
ax[1].set_title(label = "Statistics for that values of 'customer service calls',\n which are needed to draw attention ", loc = "center", pad = 35, fontweight="bold")
ax[1].xaxis.set_tick_params(labelrotation=0)
ax[1].set_xlabel(xlabel = 'customer_service_calls', fontsize = 22)

legend = plt.legend(['Clients did not leave', 'Clients left'], title = "Churn", prop = {'size':18}, loc='upper right', bbox_to_anchor=(0.38, 1.015), fancybox = True)

In [ ]:
#this is visualisation the S curve of the logistic regression 'churn ~ total_day_minutes'. 
#in the plot there is the threshold value, from which point customers are more than 50% like to churn

mask = df.loc[:,'total_day_minutes'] >= (p_y[p_y>=0.5].index[0])
fig, ax = plt.subplots(figsize =[12,6])
ax.plot(p_y, color = my_colors[0])
ax.set( xlim= [0,700])
ax.axvline(p_y[p_y>=0.5].index[0], color='black')

an_for_point = ax.annotate(s = '',
                 xy=(340, 0.52),
                 xytext=(275, 0.7),
                arrowprops=dict(facecolor='black'))
plt.ylabel(ylabel = 'Propability of churn', fontsize =20)
plt.xlabel(xlabel='total_day_charge', fontsize = 20)
string = plt.text(205, 0.73, "The threshold \n        value")
ax.set_title(label = "The S curle of the logistic regression \n 'churn ~ total_day_minutes '", loc = "center", pad = 20, fontweight="bold")

ax.scatter([349], [0.5], color='brown', s=250, marker='o')

# 6) Formulating a recommendation

In [ ]:
## Insights and recommendations from the data set

#The result of the first marketing campaign is shown that it should be drawn attention to such cities:
#Jacksonville, Orlando1, Cape Coral, Orlando2
#These cities have the biggest churn of customers, that's why these clients may leave

#For the second marketing company:

#An examination of categorical data showed that customers who signed up for an international plan had 
#questionable churn statistics. More than 40 percent of customers left the company after using this package.
#From this we can conclude that the company should pay attention to those customers who have completed the international plan 
#but have not yet left the company.

#The company also needs to focus on those customers who more often contact the support service, as there is a noticeable trend:
#the more a customer makes calls to the support service, the more likely he will leave the company

#Unfortunately, the logistic regression did not help me to identify the people 
#who would have the conditions of the poster campaign

#All in all, general amount of people that should be contacted according to my analysis: 311 